In [ ]:
import arcpy
import arcgis
import pandas as pd
import os

In [ ]:
workspace_folder = r"C:\data\Geonames"
geonames_file_path = os.path.join(workspace_folder, "cities500.csv")

In [ ]:
geonames_column_names = [
    "geonameid",
    "name",
    "ascii_name",
    "alternate_names",
    "latitude",
    "longitude",
    "feature_class",
    "feature_code",
    "country_code",
    "cc2",
    "admin1_code",
    "admin2_code",
    "admin3_code",
    "admin4_code",
    "population",
    "elevation",
    "dem",
    "timezone",
    "modification_date"
]
geonames_dtypes_dict = {
    0: int, # geonameid
    1: str,  # name
    2: str,  # asciiname
    3: str,  # alternatenames
    4: float, # latitude
    5: float, # longitude
    6: str, # feature class
    7: str, # feature code
    8: str, # country code
    9: str, # cc2
    10: str, # admin1 code
    11: str, # admin2 code
    12: str, # admin3 code
    13: str, # admin4 code
    14: int, # population
    15: float, # elevation
    16: int, # dem (digital elevation model)
    17: str, # timezone
    18: str # modification date yyyy-MM-dd
}

In [ ]:
cities500_df = pd.read_csv(geonames_file_path, sep='\t', header=None, names=geonames_column_names, dtype=geonames_dtypes_dict)
cities500_df

In [ ]:
max_rows = 100

In [ ]:
cities500_sdf = arcgis.features.GeoAccessor.from_xy(cities500_df[:max_rows], x_column='longitude', y_column='latitude')
cities500_sdf

In [ ]:
cities500_fc = cities500_sdf.spatial.to_featureclass("in_memory/cities500")
cities500_fc

In [ ]:
gp_result = arcpy.analysis.Buffer(cities500_fc, "in_memory/buffers", "45 kilometers", method="GEODESIC")
gp_result

In [ ]:
if 4 != gp_result.status:
    raise Exception("GP Tool failed!")
    
buffers_fc = gp_result[0]
buffers_sdf = arcgis.features.GeoAccessor.from_featureclass(buffers_fc)
buffers_sdf

In [ ]:
columns = ["name", "SHAPE"]

In [ ]:
gis = arcgis.gis.GIS("pro")
focus_map = gis.map("Europe")
buffers_sdf[columns].spatial.plot(focus_map)
focus_map